In [1]:
import pandas as pd
# from functions import cutDf
from datetime import datetime, timedelta
import numpy as np
import sys
sys.path.append('..')
from pyglicko2.glicko2_tests import exampleCase
from pyglicko2.glicko2 import Player
import glicko2
import time


In [2]:
matches = pd.read_csv('../Data/matches_glicko2.csv',parse_dates = ['tourney_date'], 
                      infer_datetime_format = True)

In [3]:
# matches['tourney_date'] = matches['tourney_date'].apply(
#     lambda x: pd.to_datetime(str(x), format='%Y%m%d')) # convert the tourney_date to datetime

In [4]:
for x in []:
    print(type(x))

In [5]:
players_to_instantiate = list(set([1,2]) - set([1,2]))
new_players = {player: Player() for player in players_to_instantiate}

In [65]:
def epochG(matches, players_dict,cutoff_date):
    '''Take in a dataframe with matches, a list of players, a dictionary of players with the elements
    of player list as keys and an instance of the glicko2 class Player() as the value.
    This function wll return the updated players_list and players_dict.'''
    
    players_list = list(players_dict.keys())
    players = [p for p in np.append(matches['winner_id'].unique(),\
                                                matches['loser_id'].unique())]
    # instantiate player class for players not yet instantiated
    players_to_instantiate = list(set(players) - set(players_list))
    
    new_players = {player: Player() for player in players_to_instantiate}
    # print(type(new_players))
    players_list = list(set(players).union(set(players_list)))

    # update list of instantiated players
    # update dictionary of instantiated players
    players_dict.update(new_players)
    # print(type(players_dict))
    # determine who competed and who didn't
    players_dnc = list(set(players_list) - set(players))
    players_compete = players
    # print(f'{players_compete=}')
    # update rating deviation for players who didn't compete
    for player in players_dnc:
        players_dict[player].did_not_compete()
    # fill dictionary with (wins,losses), a tuple of list of opponents in wins and losses for each match
    results = {}
    # update players who did compete
    for player in players_compete:
        # get id of players they beat
        wins = [winner for winner in matches[matches['winner_id']==player]['loser_id']]
        # get id of players they lost to
        losses = [loser for loser in matches[matches['loser_id']== player]['winner_id']]
        # get opponents' rating and rd
        losses_rating = [players_dict[loss].getRating() for loss in losses]
        losses_rd = [players_dict[loss].getRd() for loss in losses]
        wins_rating = [players_dict[win].getRating() for win in wins]
        wins_rd = [players_dict[win].getRd() for win in wins]
        outcomes = np.append(np.ones(len(wins)),np.zeros(len(losses)))
        opponent_rating = wins_rating + losses_rating
        # print(f'{opponent_rating=}')
        opponent_rd = wins_rd + losses_rd
        results[player] = (opponent_rating, opponent_rd, outcomes)
    
    ratings_timestamp = {}
    # update players
    for player in list(results.keys()):
        (rating_list, RD_list, outcome_list) = results[player]
        # print(f'{rating_list=}, {RD_list=}, {outcome_list=}')
        players_dict[player].update_player(rating_list, RD_list, outcome_list)
        ratings_timestamp[(player,cutoff_date)] = players_dict[player].getRating()
        
    return players_dict,ratings_timestamp
    # opponent_indices = matches[matches['loser_id'].str==player '']


In [67]:
epochG(matches[1:10],{},matches.iloc[10]['tourney_date'])

({113953: <pyglicko2.glicko2.Player at 0x7fc8a00089d0>,
  114146: <pyglicko2.glicko2.Player at 0x7fc8a0008970>,
  113987: <pyglicko2.glicko2.Player at 0x7fc8a0008850>,
  114147: <pyglicko2.glicko2.Player at 0x7fc8a0008880>,
  114149: <pyglicko2.glicko2.Player at 0x7fc8a00084c0>,
  114154: <pyglicko2.glicko2.Player at 0x7fc8a00085e0>,
  113963: <pyglicko2.glicko2.Player at 0x7fc8a0008430>,
  114158: <pyglicko2.glicko2.Player at 0x7fc8a0008100>,
  113999: <pyglicko2.glicko2.Player at 0x7fc8a0008250>,
  113976: <pyglicko2.glicko2.Player at 0x7fc8a0008730>,
  113982: <pyglicko2.glicko2.Player at 0x7fc8a00087f0>},
 {(113987, Timestamp('1877-07-09 00:00:00')): 1799.6258356900184,
  (113963, Timestamp('1877-07-09 00:00:00')): 1599.8752635134226,
  (113999, Timestamp('1877-07-09 00:00:00')): 1599.8752635134226,
  (114158, Timestamp('1877-07-09 00:00:00')): 1500.0,
  (114149, Timestamp('1877-07-09 00:00:00')): 1662.3108949741131,
  (113953, Timestamp('1877-07-09 00:00:00')): 1337.6891050258869,

In [70]:
def epochsG(match_history, interval_length = 365):
    '''Calculate the ending rating for each player with the lengh of each epoch being
    a funtion of the interval_length
    
    Next iterations: generate a rating history indicating the rating of each player each time that
    the rating updates.
    '''
    # check if the match_history is empty, as if it is the function call will not complete.
    if match_history.empty:
        return "Try again with a non-empty match history!"
    max_date = max(match_history['tourney_date']) # maximum date in the records
    min_date = min(match_history['tourney_date']) # minimum date in the records
    date_range = range(0,(max_date-min_date).days + 1,interval_length) # days from date of first
    # match in increments of the interval (default 365).  This is the length of each epoch. 
    epoch_cutoffs = [min_date + timedelta(days = x) for x in date_range] # The times that
    # divide the matches into each epoch.
    epoch_ranges = zip(epoch_cutoffs[0:-2],epoch_cutoffs[1:-1]) # each epoch will include matches
    # greater than or equal to the first element, less than the second element for the zip
    # generator's respective item.
    # print([r for r in epoch_ranges])
    players_dict = {} # instantiate the dictionary that will hold a Player() class for each player.
    ratings_history = {}

    # iteratively re-update for each epoch
    for period in epoch_ranges:
        # rating_ period is the df of matches that fall within a given epoch
        rating_period = match_history[(match_history['tourney_date']>=period[0])&
                                      (match_history['tourney_date']<period[1])]
        # If the rating period is empty, then adjust the rating deviation of the players.
        if not rating_period.empty:
            players_dict,ratings_timestamp = epochG(rating_period,players_dict,period[1])
            ratings_history.update(ratings_timestamp)
        else:
            for player in players_dict:
                players_dict[player].did_not_compete()
    # get the final rating period updates (for matches in the final 365 to 729 days).
    rating_period = match_history[match_history['tourney_date']>=epoch_cutoffs[-1]]
    
    players_dict,ratings_timestamp = epochG(rating_period,players_dict,max_date)
    ratings_history.update(ratings_timestamp)
    return players_dict, ratings_history

In [71]:
%%time
start = time.time()
pd,rh = epochsG(matches[256_000:257_000],90)
compute_time = time.time()-start
print(compute_time)


0.27733492851257324
CPU times: user 275 ms, sys: 3.23 ms, total: 278 ms
Wall time: 277 ms


In [72]:
rh

{(101736, Timestamp('1998-02-07 00:00:00')): 1669.3438572517261,
 (101601, Timestamp('1998-02-07 00:00:00')): 1427.8705497048709,
 (101889, Timestamp('1998-02-07 00:00:00')): 1556.4479476281233,
 (103720, Timestamp('1998-02-07 00:00:00')): 1626.6648615967056,
 (101727, Timestamp('1998-02-07 00:00:00')): 1601.8269171897591,
 (102257, Timestamp('1998-02-07 00:00:00')): 1731.8366983918638,
 (102223, Timestamp('1998-02-07 00:00:00')): 1500.0,
 (101441, Timestamp('1998-02-07 00:00:00')): 1840.5322689713878,
 (102148, Timestamp('1998-02-07 00:00:00')): 1572.1294502951291,
 (101964, Timestamp('1998-02-07 00:00:00')): 1782.239810063506,
 (101820, Timestamp('1998-02-07 00:00:00')): 1601.8269171897591,
 (101463, Timestamp('1998-02-07 00:00:00')): 1716.3883743890067,
 (101723, Timestamp('1998-02-07 00:00:00')): 1500.0,
 (108812, Timestamp('1998-02-07 00:00:00')): 1283.6116256109933,
 (102817, Timestamp('1998-02-07 00:00:00')): 1859.911026374275,
 (102687, Timestamp('1998-02-07 00:00:00')): 1546.3

In [13]:
%time
type(matches['tourney_date'][0])

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 3.1 µs


pandas._libs.tslibs.timestamps.Timestamp

In [14]:
epoch(matches[matches['tourney_date'] <= "1879-07-09"],{})

{113953: <pyglicko2.glicko2.Player at 0x7fc8e348a2e0>,
 113954: <pyglicko2.glicko2.Player at 0x7fc8e348a3a0>,
 113955: <pyglicko2.glicko2.Player at 0x7fc8e348a370>,
 113956: <pyglicko2.glicko2.Player at 0x7fc8e348a220>,
 113957: <pyglicko2.glicko2.Player at 0x7fc8e348a1c0>,
 113958: <pyglicko2.glicko2.Player at 0x7fc8e348a160>,
 113959: <pyglicko2.glicko2.Player at 0x7fc8e348a100>,
 113960: <pyglicko2.glicko2.Player at 0x7fc8e348a0a0>,
 113961: <pyglicko2.glicko2.Player at 0x7fc8e348a310>,
 113962: <pyglicko2.glicko2.Player at 0x7fc8e348a2b0>,
 113963: <pyglicko2.glicko2.Player at 0x7fc8e348a460>,
 113964: <pyglicko2.glicko2.Player at 0x7fc8e348a4c0>,
 113965: <pyglicko2.glicko2.Player at 0x7fc8e348a520>,
 113966: <pyglicko2.glicko2.Player at 0x7fc8e348a580>,
 113967: <pyglicko2.glicko2.Player at 0x7fc8e348a5e0>,
 113968: <pyglicko2.glicko2.Player at 0x7fc8e348a640>,
 113969: <pyglicko2.glicko2.Player at 0x7fc8e348a6a0>,
 113970: <pyglicko2.glicko2.Player at 0x7fc8e348a700>,
 113971: <

In [15]:
matches[matches['tourney_date'] <= "1879-07-09"]

,tourney_date,winner_id,loser_id,tourney_id
0,1877-07-09,113987,114149,1877-540
1,1877-07-09,113987,113999,1877-540
2,1877-07-09,113987,113963,1877-540
3,1877-07-09,113963,114158,1877-540
4,1877-07-09,113999,113953,1877-540
...,...,...,...,...
95,1879-07-07,114008,114009,1879-540
96,1879-07-07,113981,114010,1879-540
97,1879-07-07,114011,114012,1879-540
98,1879-07-07,114013,114014,1879-540
